# Create Phenopackets file for (synthetic) EATRIS-Plus phenotype data

Define input and output file paths

In [ ]:
import os
# synthetic test data set
datadir = "../data"
# input files
phenotype_data_file = os.path.join(datadir, "example", "synthetic_phenotype.txt")
# ontology term mappings
phenotype_variables_file = os.path.join(datadir, "ontology_terms", "phenotype_variables.csv")
phenotype_values_files = {
    "param23795": os.path.join(datadir, "ontology_terms", "ABO_blood_group_values.csv"),
    "param23796": os.path.join(datadir, "ontology_terms", "Rh_blood_group_values.csv"),
    "param18133": os.path.join(datadir, "ontology_terms", "smoking_values.csv"),
    "param23797": os.path.join(datadir, "ontology_terms", "PhenotypicFeature_values.csv")}
# ouput files
phenopackets_json_file = os.path.join(datadir, "example", "phenopacket.json")

Read input files

In [ ]:
import pandas as pd
phenotype_data = pd.read_table(phenotype_data_file)
phenotype_variables = pd.read_csv(phenotype_variables_file)
phenotype_values = {p: pd.read_csv(f) for p, f in 
                   phenotype_values_files.items()}

In [ ]:
phenotype_data.head()

In [ ]:
phenotype_variables

In [ ]:
phenotype_values

Definition of EATRIS-Plus phenopacket class

In [ ]:
import phenopackets 
from datetime import date
from calendar import timegm
from google.protobuf.timestamp_pb2 import Timestamp

AVERAGE_YEAR_DAYS = 365.2425

class EATRISPlusPhenopacket():
    def __init__(self, data, variables, values):
        self.data = data
        self.variables = variables
        self.values = values

    def _get_timestamp(self, date_str):
        # get date of birth and sampling date timestamps; 
        # seconds since Unix epoch
        # https://phenopacket-schema.readthedocs.io/en/latest/timestamp.html
        y, m, d = tuple([int(i) for i in date_str.split("-")])
        timestamp = Timestamp(seconds = timegm(date(y, m, d).timetuple()))
        return timestamp

    def _get_age(self, years):
        # create age object
        # https://phenopacket-schema.readthedocs.io/en/latest/age.html
        age = phenopackets.TimeElement(
            age = phenopackets.Age(iso8601duration = "P{0}Y".format(years)))
        return age

    def _create_measurement(self, param):
        # https://phenopacket-schema.readthedocs.io/en/latest/measurement.html
        var_info = self.variables.loc[self.variables["column_name"] == param,]
        if param not in self.values:
            measurement = phenopackets.Measurement(
                description = var_info["column_label"].values[0],
                assay = phenopackets.OntologyClass(
                    id = var_info["Term CURIE"].values[0], 
                    label = var_info["Term label"].values[0]),
                value = phenopackets.Value(
                    quantity = phenopackets.Quantity(
                        unit = phenopackets.OntologyClass(
                            id = var_info["Unit term CURIE"].values[0], 
                            label = var_info["Unit term label"].values[0]),
                        value = self.data[param]))
                        #reference_range = phenopackets.ReferenceRange()
                #time_observed = self.sampling_age_iso8601
            #procedure
            )
        else: # categorical values
            try:
                value_reported = self.data[param]
                if pd.isnull(value_reported):
                    value_info = self.values[param].loc[
                        pd.isnull(self.values[param]["value_reported"]),]
                else:
                    value_info = self.values[param].loc[
                        self.values[param]["value_reported"] == value_reported,]
                measurement = phenopackets.Measurement(
                    description = var_info["column_label"].values[0],
                    assay = phenopackets.OntologyClass(
                        id = var_info["Term CURIE"].values[0], 
                        label = var_info["Term label"].values[0]),
                    value = phenopackets.Value(
                        ontology_class = phenopackets.OntologyClass(
                            id = value_info["Term CURIE"].values[0], 
                            label = value_info["Term label"].values[0]))
                )
            except:
                print(param, value_reported)
                raise
        return measurement

    def _create_phenotypic_feature(self, param):
        var_info = self.variables.loc[self.variables["column_name"] == param,]
        value_reported = self.data[param]
        if pd.isnull(value_reported):
            raise ValueError("Value for PhenotypicFeature cannot be NULL/NAN")
        value_info = self.values[param].loc[
            self.values[param]["value_reported"] == value_reported,]
        phenotypic_feature = phenopackets.PhenotypicFeature(
            description = value_info["value_reported"].values[0],
            type = phenopackets.OntologyClass(
                id = value_info["Term CURIE"].values[0], 
                label = value_info["Term label"].values[0])
            )
        return phenotypic_feature
    
    @property
    def identifier(self):
        ID = self.data["ID Patient"]
        return ID

    @property
    def phenotypic_sex(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/sex.html
        return self.data["param16320"].upper()

    @property
    def date_of_birth_timestamp(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/timestamp.html
        return self._get_timestamp(self.data["param16318"])

    @property
    def sampling_date_timestamp(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/timestamp.html
        return self._get_timestamp(self.data["param6620"])

    @property
    def sampling_age_years(self):
        # calculate from date of birth and sampling date
        b = self.date_of_birth_timestamp.ToDatetime() 
        s = self.sampling_date_timestamp.ToDatetime() 
        #years = s.year - b.year - ((s.month, s.day) < (b.month, b.day))
        years = round((s-b).days / AVERAGE_YEAR_DAYS)
        # compare to reported age
        years_reported = self.data["param23794"]
        if years != years_reported:
            raise ValueError("""Age (param23794) is not equal rounded difference 
            between date of birth (param16318) and sampling date (param6620)""")
        return years

    @property
    def sampling_age_iso8601(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/age.html
        return self._get_age(self.sampling_age_years)

    @property
    def individual(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/individual.html
        _individual = phenopackets.Individual(
            id = self.identifier,
            # NOTE: date of birth should not be included due to privacy
            #date_of_birth = self.date_of_birth_timestamp, 
            # NOTE: time_at_last_encounter can be sampling age or sampling date
            time_at_last_encounter = self.sampling_age_iso8601, 
            #vital_status = phenopackets.VitalStatus(status = "ALIVE"),
            sex = self.phenotypic_sex,
            #karyotypic_sex = phenopackets.KaryotypicSex(),
            taxonomy = phenopackets.OntologyClass(
                id = "NCBITaxon:9606", label = "Homo sapiens"))
        return _individual

    @property
    def individual_measurements(self):
        params = self.variables.loc[
            (self.variables["phenopackets_building_block"] == "Measurement") &
            (self.variables["part_of_phenopackets_building_block"] == 
                "Individual"), ]
        measurements = [
            self._create_measurement(p) for p in params["column_name"]]
        return measurements

    @property
    def biosample_measurements(self):
        params = self.variables.loc[
            (self.variables["phenopackets_building_block"] == "Measurement") &
            (self.variables["part_of_phenopackets_building_block"] == 
                "Biosample"), ]
        measurements = [
            self._create_measurement(p) for p in params["column_name"]]
        return measurements

    @property
    def biosample(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/biosample.html
        _biosample = phenopackets.Biosample(
            id = self.identifier,
            individual_id = self.identifier,
            description = "Blood sample",
            sampled_tissue = phenopackets.OntologyClass(
                id = "NCIT:C17610", label = "Blood Sample"),
            measurements = self.biosample_measurements,
            time_of_collection = self.sampling_age_iso8601
        )
        return _biosample

    @property
    def phenotypic_features(self):
        params = self.variables.loc[
            (self.variables["phenopackets_building_block"] == "PhenotypicFeature") &
            (self.variables["part_of_phenopackets_building_block"] == 
                "Individual"), ]
        _phenotypic_features = [
            self._create_phenotypic_feature(p) for p in params["column_name"]
            if not pd.isnull(self.data[p])]
        return _phenotypic_features
    
    @property
    def phenopacket(self):
        # https://phenopacket-schema.readthedocs.io/en/latest/phenopacket.html
        _phenopacket = phenopackets.Phenopacket(
            id = self.identifier,
            subject = self.individual,
            phenotypic_features = self.phenotypic_features,
            measurements = self.individual_measurements,
            biosamples = [self.biosample]
        )
        return _phenopacket

Create example phenopacket for first row

In [ ]:
row = phenotype_data.iloc[1,]
print(row)
p = EATRISPlusPhenopacket(row, phenotype_variables, phenotype_values)
p.phenopacket

Create phenopackets json file for entire data set

In [ ]:
from google.protobuf.json_format import MessageToJson

with open(phenopackets_json_file, "w") as outfile:
    all_phenopackets = []
    for index, row in phenotype_data.iterrows():
        p = EATRISPlusPhenopacket(
            row, phenotype_variables, phenotype_values).phenopacket
        all_phenopackets.append(p)
    cohort = phenopackets.Cohort(
        id = "Synthetic data set",
        description = "Synthetic data set based on fields present in the EATRIS-Plus healthy multi-omics demonstrator cohort",
        members = all_phenopackets,
        meta_data = phenopackets.MetaData(
            created = Timestamp(seconds = timegm(date.today().timetuple())),
            created_by = "Anna Niehues",
            resources = [
                phenopackets.Resource(
                    id = "ncbitaxon",
                    name = "NCBI organismal classification",
                    url = "https://github.com/obophenotype/ncbitaxon",
                    version = "2022-08-18",
                    namespace_prefix = "NCBITaxon",
                    iri_prefix = "http://purl.obolibrary.org/obo/NCBITaxon_"
                ),
                phenopackets.Resource(
                    id = "ncit",
                    name = "NCI Thesaurus OBO Edition",
                    url = "https://github.com/NCI-Thesaurus/thesaurus-obo-edition",
                    version = "2022-08-19",
                    namespace_prefix = "NCIT",
                    iri_prefix = "http://purl.obolibrary.org/obo/NCIT_"
                ),
                phenopackets.Resource(
                    id = "efo",
                    name = "Experimental Factor Ontology",
                    url = "https://www.ebi.ac.uk/efo/",
                    version = "v3.47.0",
                    namespace_prefix = "EFO",
                    iri_prefix = "http://www.ebi.ac.uk/efo/EFO_"
                ),
                phenopackets.Resource(
                    id = "uo",
                    name = "Units of measurement ontology",
                    url = "https://github.com/bio-ontology-research-group/unit-ontology",
                    version = "2020-03-10",
                    namespace_prefix = "UO",
                    iri_prefix = "http://purl.obolibrary.org/obo/UO_"
                ),
                phenopackets.Resource(
                    id = "loinc",
                    name = "Logical Observation Identifier Names and Codes",
                    url = "https://loinc.org/",
                    version = "2.73",
                    namespace_prefix = "LOINC",
                    iri_prefix = "https://loinc.org/"
                ),
                phenopackets.Resource(
                    id = "doid",
                    name = "Human Disease Ontology",
                    url = "https://github.com/DiseaseOntology/HumanDiseaseOntology",
                    version = "2022-09-29",
                    namespace_prefix = "DOID",
                    iri_prefix = "http://purl.obolibrary.org/obo/DOID_"
                )
            ],
            phenopacket_schema_version = "2.0"
        )
    )
    outfile.write(MessageToJson(cohort))
    